In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/Usuario/Documents/Cursos Udemy/urls2.csv")
df.head(5)

,urls
0,https://www.lighting.philips.com/content/b2b-p...
1,https://www.lighting.philips.com/content/b2b-p...
2,https://www.lighting.philips.com/content/b2b-p...
3,https://www.lighting.philips.com/content/b2b-p...
4,https://www.lighting.philips.com/content/b2b-p...


In [6]:
##### pruebas
import asyncio
import nest_asyncio
import csv
from playwright.async_api import async_playwright

nest_asyncio.apply()

# Carga tu CSV como DataFrame
urls = ["https://www.lighting.philips.com.ph/application-areas/retail-hospitality/fruits-vegetables","https://www.lighting.philips.nl/toepassingsgebieden/public-spaces/museums"]

async def detectar_form_marketo_con_iframes(page, url):
    print(f"\n🌐 Procesando URL: {url}")

    form_detectado = False

    def intercept_request(request):
        nonlocal form_detectado
        if "form/getForm" in request.url:
            print(f"✅ Petición Marketo detectada: {request.url}")
            form_detectado = True

    page.on("request", intercept_request)
    page.on("frameattached", lambda frame: frame.on("request", intercept_request))

    try:
        await page.goto(url, timeout=20000)
        await page.wait_for_timeout(3000)
    except Exception as e:
        print(f"❌ Error cargando la página: {e}")
        return False

    try:
        await page.evaluate("""
            () => {
                const buttons = document.querySelectorAll("button");
                for (const btn of buttons) {
                    if (btn.innerText.toLowerCase().includes("accept") || btn.innerText.toLowerCase().includes("allow")) {
                        btn.click();
                        break;
                    }
                }
            }
        """)
        print("✔️ Cookies aceptadas")
        await page.wait_for_timeout(2000)
    except:
        print("❌ No se aceptaron cookies")

    try:
        await page.evaluate("""
            () => {
                const overlay = document.querySelector(".truste_overlay");
                if (overlay) {
                    overlay.remove();
                    console.log("🧼 Overlay eliminado");
                }
            }
        """)
    except Exception as e:
        print(f"⚠️ No se pudo eliminar overlay: {e}")

    try:
        ctas = await page.query_selector_all('[class*="cta__element cta__element--single button cta__element--primary"]')
        print(f"🔍 Detectados {len(ctas)} elementos con 'cta__element' en la clase")

        for i, el in enumerate(ctas):
            if form_detectado:
                print("⛔ Formulario Marketo ya detectado. Se detiene el clic de CTAs.")
                break
            try:
                await el.scroll_into_view_if_needed()
                await el.click(timeout=5000)
                print(f"🟢 Clic en CTA #{i+1}")
                await page.wait_for_timeout(4000)
            except Exception as e:
                print(f"⚠️ No se pudo clicar CTA #{i+1}: {e}")

        # IF: Si no se detectó el formulario después de ctas normales, probar con los 'cta-component--modal-button'
        if not form_detectado:
            print("🔄 No se detectó formulario tras clicar en cta__element. Buscando botones modales...")
            modal_ctas = await page.query_selector_all('[class*="cta-component__link button"]')
            print(f"🔍 Detectados {len(modal_ctas)} botones modales")

            for j, modal_el in enumerate(modal_ctas):
                if form_detectado:
                    print("⛔ Formulario Marketo detectado durante clic en botones modales.")
                    break
                try:
                    await modal_el.scroll_into_view_if_needed()
                    await modal_el.click(timeout=5000)
                    print(f"🟡 Clic en botón modal #{j+1}")
                    await page.wait_for_timeout(4000)
                except Exception as e:
                    print(f"⚠️ No se pudo clicar botón modal #{j+1}: {e}")
    except Exception as e:
        print(f"❌ Error buscando los CTAs: {e}")

    if form_detectado:
        print("✅ ¡Se detectó el formulario Marketo!")
    else:
        print("❌ No se detectó ninguna petición a form/getForm")

    page.remove_listener("request", intercept_request)

    return form_detectado

async def main():
    resultados = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        for url in urls:
            resultado = await detectar_form_marketo_con_iframes(page, url)
            resultados.append({"url": url, "form_marketo_encontrado": resultado})

        await browser.close()

if __name__ == "__main__":
    asyncio.run(main())



🌐 Procesando URL: https://www.lighting.philips.com.ph/application-areas/retail-hospitality/fruits-vegetables
✔️ Cookies aceptadas
🔍 Detectados 2 elementos con 'cta__element' en la clase
🟢 Clic en CTA #1
✅ Petición Marketo detectada: https://485-ikz-407.mktoweb.com/index.php/form/getForm?munchkinId=485-IKZ-407&form=1113&url=https%3A%2F%2Fpages.information.signify.com%2Fmulti-gated&callback=jQuery37108995266559579183_1752914115894&_=1752914115895
✅ Petición Marketo detectada: https://pages.information.signify.com/index.php/form/getForm?munchkinId=485-IKZ-407&form=1298
⛔ Formulario Marketo ya detectado. Se detiene el clic de CTAs.
✅ ¡Se detectó el formulario Marketo!

🌐 Procesando URL: https://www.lighting.philips.nl/toepassingsgebieden/public-spaces/museums
✔️ Cookies aceptadas
🔍 Detectados 0 elementos con 'cta__element' en la clase
🔄 No se detectó formulario tras clicar en cta__element. Buscando botones modales...
🔍 Detectados 2 botones modales
⚠️ No se pudo clicar botón modal #1: Elem

In [9]:
## Prueba definitiva. Falta cambiar las urls a analizar + la parte de descarga de resultados
## Contiene el IF para clicar dos clases de botón. Y la funcionalidad para no tener muchas pestañas abiertas

import asyncio
import nest_asyncio
import csv
from playwright.async_api import async_playwright

nest_asyncio.apply()

# Lista de URLs de prueba
urls = [
    "https://www.lighting.philips.com.ph/application-areas/retail-hospitality/fruits-vegetables",
    "https://www.lighting.philips.nl/toepassingsgebieden/public-spaces/museums",
    "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/support/cases/airports/schiphol-airport",
    "https://www.lighting.philips.com/content/b2b-philips-lighting/sg/en/support/cases/airports/schiphol-airport",
    "https://www.lighting.philips.com/content/b2b-philips-lighting/sg/en/products/highlighted-products/ledstrips"
]

async def detectar_form_marketo_con_iframes(page, url):
    print(f"\n🌐 Procesando URL: {url}")

    form_detectado = False

    def intercept_request(request):
        nonlocal form_detectado
        if "form/getForm" in request.url:
            print(f"✅ Petición Marketo detectada: {request.url}")
            form_detectado = True

    page.on("request", intercept_request)
    page.on("frameattached", lambda frame: frame.on("request", intercept_request))

    try:
        await page.goto(url, timeout=20000)
        await page.wait_for_timeout(3000)
    except Exception as e:
        print(f"❌ Error cargando la página: {e}")
        return False

    try:
        await page.evaluate("""
            () => {
                const buttons = document.querySelectorAll("button");
                for (const btn of buttons) {
                    if (btn.innerText.toLowerCase().includes("accept") || btn.innerText.toLowerCase().includes("allow")) {
                        btn.click();
                        break;
                    }
                }
            }
        """)
        print("✔️ Cookies aceptadas")
        await page.wait_for_timeout(2000)
    except:
        print("❌ No se aceptaron cookies")

    try:
        await page.evaluate("""
            () => {
                const overlay = document.querySelector(".truste_overlay");
                if (overlay) {
                    overlay.remove();
                    console.log("🧼 Overlay eliminado");
                }
            }
        """)
    except Exception as e:
        print(f"⚠️ No se pudo eliminar overlay: {e}")

    try:
        ctas = await page.query_selector_all('[class*="cta__element cta__element--single button cta__element--primary"]')
        print(f"🔍 Detectados {len(ctas)} elementos con 'cta__element' en la clase")

        for i, el in enumerate(ctas):
            if form_detectado:
                print("⛔ Formulario Marketo ya detectado. Se detiene el clic de CTAs.")
                break
            try:
                await el.scroll_into_view_if_needed()
                await el.click(timeout=5000)
                print(f"🟢 Clic en CTA #{i+1}")
                await page.wait_for_timeout(4000)
            except Exception as e:
                print(f"⚠️ No se pudo clicar CTA #{i+1}: {e}")

        if not form_detectado:
            print("🔄 No se detectó formulario tras clicar en cta__element. Buscando botones modales...")
            modal_ctas = await page.query_selector_all('[class*="cta-component__link button"]')
            print(f"🔍 Detectados {len(modal_ctas)} botones modales")

            for j, modal_el in enumerate(modal_ctas):
                if form_detectado:
                    print("⛔ Formulario Marketo detectado durante clic en botones modales.")
                    break
                try:
                    await modal_el.scroll_into_view_if_needed()
                    await modal_el.click(timeout=5000)
                    print(f"🟡 Clic en botón modal #{j+1}")
                    await page.wait_for_timeout(4000)
                except Exception as e:
                    print(f"⚠️ No se pudo clicar botón modal #{j+1}: {e}")
    except Exception as e:
        print(f"❌ Error buscando los CTAs: {e}")

    if form_detectado:
        print("✅ ¡Se detectó el formulario Marketo!")
    else:
        print("❌ No se detectó ninguna petición a form/getForm")

    page.remove_listener("request", intercept_request)
    return form_detectado

async def main():
    resultados = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        # 🧷 Mantener primera pestaña abierta (aunque no se use)
        primera_pestana = await context.new_page()
        paginas_abiertas = [primera_pestana]

        for url in urls:
            nueva_pestana = await context.new_page()
            paginas_abiertas.append(nueva_pestana)

            if len(paginas_abiertas) > 2:
                pagina_a_cerrar = paginas_abiertas.pop(1)  # Cierra la anterior a la actual
                await pagina_a_cerrar.close()

            resultado = await detectar_form_marketo_con_iframes(nueva_pestana, url)
            resultados.append({"url": url, "form_marketo_encontrado": resultado})

        await browser.close()

if __name__ == "__main__":
    asyncio.run(main())



🌐 Procesando URL: https://www.lighting.philips.com.ph/application-areas/retail-hospitality/fruits-vegetables
✔️ Cookies aceptadas
🔍 Detectados 2 elementos con 'cta__element' en la clase
🟢 Clic en CTA #1
✅ Petición Marketo detectada: https://485-ikz-407.mktoweb.com/index.php/form/getForm?munchkinId=485-IKZ-407&form=1113&url=https%3A%2F%2Fpages.information.signify.com%2Fmulti-gated&callback=jQuery37105836420583158474_1752914843532&_=1752914843533
✅ Petición Marketo detectada: https://pages.information.signify.com/index.php/form/getForm?munchkinId=485-IKZ-407&form=1298
⛔ Formulario Marketo ya detectado. Se detiene el clic de CTAs.
✅ ¡Se detectó el formulario Marketo!

🌐 Procesando URL: https://www.lighting.philips.nl/toepassingsgebieden/public-spaces/museums
✔️ Cookies aceptadas
🔍 Detectados 0 elementos con 'cta__element' en la clase
🔄 No se detectó formulario tras clicar en cta__element. Buscando botones modales...
🔍 Detectados 2 botones modales
⚠️ No se pudo clicar botón modal #1: Elem

In [19]:
## Código final
## Contiene el IF para clicar dos clases de botón. Y la funcionalidad para no tener muchas pestañas abiertas

import asyncio
import nest_asyncio
import csv
from playwright.async_api import async_playwright

nest_asyncio.apply()

# Lista de URLs: editar el listado que toma las filas del DF

async def detectar_form_marketo_con_iframes(page, url):
    print(f"\n🌐 Procesando URL: {url}")

    form_detectado = False

    def intercept_request(request):
        nonlocal form_detectado
        if "form/getForm" in request.url:
            print(f"✅ Petición Marketo detectada: {request.url}")
            form_detectado = True

    page.on("request", intercept_request)
    page.on("frameattached", lambda frame: frame.on("request", intercept_request))

    try:
        await page.goto(url, timeout=20000)
        await page.wait_for_timeout(3000)
    except Exception as e:
        print(f"❌ Error cargando la página: {e}")
        return False

    try:
        await page.evaluate("""
            () => {
                const buttons = document.querySelectorAll("button");
                for (const btn of buttons) {
                    if (btn.innerText.toLowerCase().includes("accept") || btn.innerText.toLowerCase().includes("allow")) {
                        btn.click();
                        break;
                    }
                }
            }
        """)
        print("✔️ Cookies aceptadas")
        await page.wait_for_timeout(2000)
    except:
        print("❌ No se aceptaron cookies")

    try:
        await page.evaluate("""
            () => {
                const overlay = document.querySelector(".truste_overlay");
                if (overlay) {
                    overlay.remove();
                    console.log("🧼 Overlay eliminado");
                }
            }
        """)
    except Exception as e:
        print(f"⚠️ No se pudo eliminar overlay: {e}")

    
    
    try:
        # Primera tanda de CTAs
        ctas = await page.query_selector_all('[class*="cta__element cta__element--single button cta__element--primary"]')
        print(f"🔍 Detectados {len(ctas)} elementos con clase 'cta__element cta__element--single button cta__element--primary'")

        for i, el in enumerate(ctas):
            if form_detectado:
                print("⛔ Formulario Marketo ya detectado. Se detiene el clic de CTAs.")
                break
            try:
                await el.scroll_into_view_if_needed()
                await el.click(timeout=5000)
                print(f"🟢 Clic en CTA #{i+1}")
                await page.wait_for_timeout(4000)
            except Exception as e:
                print(f"⚠️ No se pudo clicar CTA #{i+1}: {e}")

        # Segunda tanda de CTAs, solo si aún no se detectó el formulario
        if not form_detectado:
            alt_ctas = await page.query_selector_all('[class*="button cta__element cta__element--single cta__element--primary"]')
            print(f"🔍 Detectados {len(alt_ctas)} elementos con clase 'button cta__element cta__element--single cta__element--primary'")

            for i, el in enumerate(alt_ctas):
                if form_detectado:
                    print("⛔ Formulario Marketo ya detectado. Se detiene el clic de CTAs alternativos.")
                    break
                try:
                    await el.scroll_into_view_if_needed()
                    await el.click(timeout=5000)
                    print(f"🟢 Clic en CTA alternativo #{i+1}")
                    await page.wait_for_timeout(4000)
                except Exception as e:
                    print(f"⚠️ No se pudo clicar CTA alternativo #{i+1}: {e}")

        # Botones modales si aún no hay form detectado
        if not form_detectado:
            print("🔄 No se detectó formulario tras clicar en CTAs. Buscando botones modales...")
            modal_ctas = await page.query_selector_all('[class*="cta-component__link button"]')
            print(f"🔍 Detectados {len(modal_ctas)} botones modales")

            for j, modal_el in enumerate(modal_ctas):
                if form_detectado:
                    print("⛔ Formulario Marketo detectado durante clic en botones modales.")
                    break
                try:
                    await modal_el.scroll_into_view_if_needed()
                    await modal_el.click(timeout=5000)
                    print(f"🟡 Clic en botón modal #{j+1}")
                    await page.wait_for_timeout(4000)
                except Exception as e:
                    print(f"⚠️ No se pudo clicar botón modal #{j+1}: {e}")
    except Exception as e:
        print(f"❌ Error buscando los CTAs: {e}")

    if form_detectado:
        print("✅ ¡Se detectó el formulario Marketo!")
    else:
        print("❌ No se detectó ninguna petición a form/getForm")

    page.remove_listener("request", intercept_request)
    return form_detectado

async def main():
    resultados = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        # 🧷 Mantener primera pestaña abierta (aunque no se use)
        primera_pestana = await context.new_page()
        paginas_abiertas = [primera_pestana]

        for url in urls:
            nueva_pestana = await context.new_page()
            paginas_abiertas.append(nueva_pestana)

            if len(paginas_abiertas) > 2:
                pagina_a_cerrar = paginas_abiertas.pop(1)  # Cierra la anterior a la actual
                await pagina_a_cerrar.close()

            resultado = await detectar_form_marketo_con_iframes(nueva_pestana, url)
            resultados.append({"url": url, "form_marketo_encontrado": resultado})

        await browser.close()

    # Guardar resultados en CSV
    with open("resultados_marketo100f.csv", mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["url", "form_marketo_encontrado"])
        writer.writeheader()
        writer.writerows(resultados)

    print("\n✅ Resultados guardados en 'resultados_marketo100f.csv'")

if __name__ == "__main__":
    asyncio.run(main())



🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/nl/nl/application-areas/specialist-applications/horticulture/hortiblog/vegetables-and-fruits/dynamic-led-lighting-yields-significant-savings
✔️ Cookies aceptadas
🔍 Detectados 0 elementos con clase 'cta__element cta__element--single button cta__element--primary'
🔍 Detectados 1 elementos con clase 'button cta__element cta__element--single cta__element--primary'
🟢 Clic en CTA alternativo #1
🔄 No se detectó formulario tras clicar en CTAs. Buscando botones modales...
🔍 Detectados 0 botones modales
❌ No se detectó ninguna petición a form/getForm

🌐 Procesando URL: https://www.lighting.philips.com/content/b2b-philips-lighting/nl/nl/application-areas/specialist-applications/horticulture/hortiblog/vegetables-and-fruits/what-role-far-red-light-can-play-in-the-cultivation-of-june-bearers
✔️ Cookies aceptadas
🔍 Detectados 0 elementos con clase 'cta__element cta__element--single button cta__element--primary'
🔍 Detectad

In [18]:
urls = df["urls"].iloc[100:150].tolist()

In [20]:
urls = [
    "https://www.lighting.philips.com.ph/application-areas/retail-hospitality/fruits-vegetables",
    "https://www.lighting.philips.nl/toepassingsgebieden/public-spaces/museums",
    "https://www.lighting.philips.com/content/b2b-philips-lighting/ph/en/support/cases/airports/schiphol-airport",
    "https://www.lighting.philips.com/content/b2b-philips-lighting/sg/en/support/cases/airports/schiphol-airport",
    "https://www.lighting.philips.com/content/b2b-philips-lighting/sg/en/products/highlighted-products/ledstrips"
]